In [ ]:
from pathlib import Path
import numpy as np
from snirf import Snirf
import mne
from mne_nirs.io import write_raw_snirf
from snirf import validateSnirf
import os
import snirf
from mne.io import read_raw_nirx, read_raw_snirf
from mne.preprocessing.nirs import beer_lambert_law, optical_density
from numpy.testing import assert_allclose

In [ ]:
# homer3_to_mne_compatible.py
#
#
# - Take Homer3 HbO/HbR/HbT SNIRF
# - Drop HbT channels
# - Keep measurementList as a proper pysnirf2 IndexedGroup
# - Save a new SNIRF that mne.io.read_raw_snirf can read as hbo/hbr


# ------------------------------------------------------------
# 1) INPUT / OUTPUT PATHS  -----------------------------------
# ------------------------------------------------------------
in_fname = Path(
    "/Users/divijnalge/Library/CloudStorage/OneDrive-NanyangTechnologicalUniversity/"
    "ntu/Brain Norm/Brain_Norm_Files/fNIRS preproc/BN_DY/ses-tp1arith/"
    "nirs-homer3/3cond/output_preproc_hb.snirf"
)

out_fname = in_fname.with_name(in_fname.stem + "_mne.snirf")

print(f"Input SNIRF: {in_fname}")
print(f"Will write:  {out_fname}")

# ------------------------------------------------------------
# 2) LOAD SNIRF  ---------------------------------------------
# ------------------------------------------------------------
snirf = Snirf(str(in_fname))

# Assume single nirs element and single data block (true for Homer3 export)
nirs = snirf.nirs[0]
data = nirs.data[0]
ml = data.measurementList   # IMPORTANT: this is an IndexedGroup, not a list

# ------------------------------------------------------------
# 3) INSPECT dataTypeLabel AND BUILD KEEP MASK ---------------
# ------------------------------------------------------------
labels = [str(ml[i].dataTypeLabel) for i in range(len(ml))]
uniq = sorted(set(labels))
print("Unique dataTypeLabel values BEFORE:", uniq)

keep_mask = np.array(
    [(lab == "HbO") or (lab == "HbR") for lab in labels],
    dtype=bool,
)

n_total = len(labels)
n_keep = int(keep_mask.sum())

print(f"Total channels: {n_total}")
print(f"Keeping {n_keep} channels (HbO/HbR only)")
print(f"Dropping {n_total - n_keep} channels (e.g. HbT)")

# ------------------------------------------------------------
# 4) SLICE dataTimeSeries TO MATCH KEEP MASK -----------------
# ------------------------------------------------------------
ts = np.asarray(data.dataTimeSeries)
if ts.shape[1] != n_total:
    raise RuntimeError(
        f"dataTimeSeries has {ts.shape[1]} columns but measurementList has {n_total} entries"
    )

ts = ts[:, keep_mask]
data.dataTimeSeries = ts  # assign back (pysnirf2 accepts numpy arrays)

# ------------------------------------------------------------
# 5) DROP UNWANTED measurementList ENTRIES IN PLACE ----------
# ------------------------------------------------------------
# Do NOT do: data.measurementList = [...]
# Instead, delete entries from the existing IndexedGroup, from end to start.
for i in range(n_total - 1, -1, -1):
    if not keep_mask[i]:
        del ml[i]

# Sanity check
if len(ml) != n_keep:
    raise RuntimeError(
        f"After deletion, measurementList has {len(ml)} entries, expected {n_keep}"
    )

# ------------------------------------------------------------
# 6) SAVE NEW SNIRF  -----------------------------------------
# ------------------------------------------------------------
snirf.save(str(out_fname))
print("Done.")


In [ ]:
# fix_landmarks_in_snirf.py

# 1) Path to the _mne.snirf file you just created
snirf_path = Path(
    "/Users/divijnalge/Library/CloudStorage/OneDrive-NanyangTechnologicalUniversity/"
    "ntu/Brain Norm/Brain_Norm_Files/fNIRS preproc/BN_DY/ses-tp1arith/"
    "nirs-homer3/3cond/output_preproc_hb_mne.snirf"
)

print("Opening:", snirf_path)

# 2) Open file for read/write (r+)
with Snirf(str(snirf_path), "r+") as snirf:
    probe = snirf.nirs[0].probe

    print("Before:")
    print("  landmarkPos3D:", probe.landmarkPos3D)
    print("  landmarkLabels:", probe.landmarkLabels)

    # 3) Remove landmarks (let them be None / absent)
    probe.landmarkPos3D = None
    probe.landmarkLabels = None

    # 4) Save in-place
    snirf.save()

    print("After:")
    print("  landmarkPos3D:", probe.landmarkPos3D)
    print("  landmarkLabels:", probe.landmarkLabels)

print("Done cleaning landmarks.")


In [ ]:
# rewrite_snirf_with_mne_fixed.py

in_fname = (
    "/Users/divijnalge/Library/CloudStorage/OneDrive-NanyangTechnologicalUniversity/"
    "ntu/Brain Norm/Brain_Norm_Files/fNIRS preproc/BN_DY/ses-tp1arith/"
    "nirs-homer3/3cond/output_preproc_hb_mne.snirf"
)
out_clean = (
    "/Users/divijnalge/Library/CloudStorage/OneDrive-NanyangTechnologicalUniversity/"
    "ntu/Brain Norm/Brain_Norm_Files/fNIRS preproc/BN_DY/ses-tp1arith/"
    "nirs-homer3/3cond/output_preproc_hb_mne_clean.snirf"
)

print("Loading", in_fname)
raw = mne.io.read_raw_snirf(in_fname, preload=True)

# --- Fix NaN wavelengths in raw.info['chs'] ---
for ch in raw.info["chs"]:
    wl = ch["loc"][9]
    if not np.isfinite(wl):
        name = ch["ch_name"]
        # Simple rule: assign dummy wavelength based on chromophore
        if name.endswith(" hbo"):
            ch["loc"][9] = 760.0   # or your real lambda1
        elif name.endswith(" hbr"):
            ch["loc"][9] = 850.0   # or your real lambda2
        else:
            # Fallback: just pick one
            ch["loc"][9] = 760.0

# --- Now write SNIRF using MNE-NIRS ---
print("Writing", out_clean)
write_raw_snirf(raw, out_clean)

# --- Validate with pysnirf2 ---
print("Validating with pysnirf2...")
res = validateSnirf(out_clean)
res.display()



In [ ]:
# Check if .snirf file is loading and valid

file_path = "/Users/divijnalge/Library/CloudStorage/OneDrive-NanyangTechnologicalUniversity/"
    "ntu/Brain Norm/Brain_Norm_Files/fNIRS preproc/BN_DY/ses-tp1arith/"
    "nirs-homer3/3cond/output_preproc_hb_mne.snirf"

out_fname = Path(file_path)

print("Loading", out_fname)
raw_hb = mne.io.read_raw_snirf(str(out_fname), preload=True)
print(raw_hb)
print(raw_hb.get_channel_types(unique=True))

result = snirf.validateSnirf(file_path)
assert result.is_valid()
result.display()
